In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix,roc_curve, auc, roc_auc_score, accuracy_score, classification_report,\
cohen_kappa_score, f1_score
from sklearn.base import clone

from scipy.stats.mstats import gmean

In [ ]:
def _risk_score(likelihoods, y):
    '''
    This function takes the probability predictions from the model and transforms them into incidence-rate weighted
    risk scores. We bucket patients based on the percentile value predicted by the model and then calculate the rate of 
    complication for patients in that bucket. We also calculate the standard dev and 95% CI on the distribution of 
    probability values within each bucket.
    
    Parameters
    ----------
    likelihoods - numpy array of probability percentages
    y - pandas dataframe, Outcome from data
    
    Output
    ----------
    risk_values - array of weighted risk values
    '''
    bucket_bounds_by_tens = list(range(0,101,10))
    percentiles = np.percentile(likelihoods, bucket_bounds_by_tens)
    risk_values = np.copy(likelihoods)
    
    for i in range(10):
        patients = np.where((likelihoods>=percentiles[i]) & (likelihoods<=percentiles[i+1]))
        rate = y[patients[0]].mean()
        risk_values[patients[0]] = rate
    
    return risk_values

In [ ]:
def calculate_optimal_threshold(X, y, classifier, title):
    
    '''
    This function creates ROC plot on validation set for each fold in 5-fold cross-validation using training dataset 
    to visualize variation among different folds. ROC curve of the mean of 5 folds is shown in blue 
    while the chance (0.5) is shown in the diagonal dash line. Also, the optimal threshold is found 
    by taking average of the optimal threshold identified in each fold in cross-validation and returned.
    
    Reference: https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html#sphx-glr-auto-examples-model-selection-plot-roc-crossval-py
    
    Parameters
    ----------
    X: numpy array, features from training data
    y: numpy array, Outcome from training data
    classifier: Trained model, i.e. model.best_estimates_
    title: String, Target for model
    '''
    
    cv = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    optimal_thresholds = []
    classifier_temp = clone(classifier)

    for train, test in cv.split(X, y):
        probas_ = classifier_temp.fit(X.iloc[train,:], y.iloc[train]).predict_proba(X.iloc[test,:])

        # Compute ROC curve and area under the curve
        fpr, tpr, thresholds = roc_curve(y.iloc[test], probas_[:, 1])
        
        #Geometric mean
        optimal_idx = np.argmax(np.sqrt(tpr*(1-fpr)))
        optimal_threshold = thresholds[optimal_idx]
        optimal_thresholds.append(optimal_threshold)
        


    return optimal_thresholds 

In [ ]:
def print_metrics(X_test,y_test,classifier,outcome,file_name=None,file_suffix=None,matrix_color='Greens'):
    
    '''
    This function prints model performance metrics based on the test set using the calculated optimal threshold.
        
    Parameters
    ----------
    X_test: numpy array, features from test data
    y_test: numpy array, Outcome from test data
    classifer: The resulting classifer.best_estimator_ that is produced when using sklearn.model_selection.GridSearchCV
    optimal_thresholds: Numpy array, Calculated optimal thresholds
    outcome: String, Target for model
    file_name: string containing relative file path and the name of input file
    file_suffix: string containing abbreviations denoting whether the model is continuous or categorical, and peri or no peri
    matrix_color: String, Color of the confusion matrix
    
    Output
    ----------
    String: Target Name, Accuracy, AUC Score (Test), Sensitivity/True positive rate: 
    String: Specificity /True negative rate:, False positive rate: , False negative rate: 
    String: Precision: ,Geometric Mean: ,Cohen's Kappa:, F Score:
    Figure: Confusion Matrix 
    '''
    
    y_test_prob = classifier.predict_proba(X_test)[:,1]
    y_pred = classifier.predict(X_test)
    risk_values = _risk_score(y_test_prob, y_test)
    
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    
    Accuracy = round(accuracy_score(y_test, y_pred),3)
    AUC =  round(roc_auc_score(y_test, y_test_prob),3)
    

    TPR = round(tp/(tp+fn),3)
    TNR = round(tn/(tn+fp),3)
    FPR = round(fp/(tn+fp),3)
    FNR = round(fn/(tp+fn),3)
    Precision = round(tp/(tp+fp),3)
    
    geo_mean = round(gmean([TPR,TNR]),3)    
    
    kappa = round(cohen_kappa_score(y_test, y_pred, weights=None),3)
    
    f_score = round(f1_score(y_test, y_pred),3)

    metrics = [[outcome,Accuracy,TPR,TNR,FPR,FNR,TPR,TNR,AUC,f_score,geo_mean,kappa, sklearn.__version__]]
    col_names = ['Model','Accuracy','True Positive Rate','True Negative Rate','False Positive Rate','False Negative Rate',
    'Sensitivity','Specificity','AUC','F1-Score','Geometric Mean','Kappa-Statistics', 'Sklearn Version']
    model_metrics = pd.DataFrame(metrics,columns=col_names)
    
    
    df_cm = pd.DataFrame(cm, columns=[0,1], index = [0,1])
    df_cm.index.name = 'Actual'
    df_cm.columns.name = 'Predicted'
    plt.figure(figsize = (10,7))
    sns.set(font_scale=1.4)#for label size
    _ = sns.heatmap(df_cm, cmap=matrix_color, annot=True,fmt='g',annot_kws={"size": 16})
    plt.title(outcome, fontsize=12)
    
    if file_name:
        plt.savefig(file_name+'_confusion_matrix'+file_suffix)
        plt.close()
        
    return risk_values, model_metrics, AUC, f_score